# Week 8: Designing new filters

### 5 March 2018

# Goals #

After doing this lab, you should be able to:
* Use a filter design tool to make a new filter, then apply it to a sound
* Use a filter's frequency response, as well as listening, to reason about its effects
* Have hands-on experience using filters for audio processing, analysis, and synthesis

# 0.1. Are you caught up on labs?

A portfolio of your first four labs was due on 16 February. If you haven't turned that in yet, please see the assignment here: https://learn.gold.ac.uk/mod/assign/view.php?id=514281

By now, you should find it easy to:

1. Synthesize a sine wave with an arbitrary amplitude, frequency, and phase offset, then play that wave back. If you don't know how to to this, revisit week 3's lab (https://learn.gold.ac.uk/mod/resource/view.php?id=512068)
2. Compute and plot an FFT of an arbitrary 1-dimensional signal (e.g. an audio wave). If you don't know how to do this, revisit week 4's lab (https://learn.gold.ac.uk/mod/resource/view.php?id=512185)
3. Use the spectrum computed by the FFT to reason about the frequencies in a sound (week 5: https://learn.gold.ac.uk/mod/resource/view.php?id=513559)
4. Use the `convolution` function to apply an effect to a sound, by convolving the sound with an impulse response (Lab 6: https://learn.gold.ac.uk/mod/resource/view.php?id=516498 and week 7: https://learn.gold.ac.uk/mod/resource/view.php?id=516499)

__Don't forget that you may be asked about using Jupyter and Python for this type of synthesis and analysis on the exam, so now is the time to try it out and ask for help if you have problems!__

# 0.2 Load audio files into Python variables #

(song1.wav is the same one you used last week, but song2.wav is new)


a. Download the following audio files and store them in the same directory as this lab file:
* http://www.doc.gold.ac.uk/~mas01rf/PMC2014-15/IPython/lab13/song1.wav
* http://www.doc.gold.ac.uk/~mas01rf/PMC2016-17/lab17/song2.wav

These are free sounds downloaded from

http://freemusicarchive.org/music/Jahzzar/Travellers_Guide/Siesta

and

http://freemusicarchive.org/music/Black_Ant/Free_Beats_Sel_3/Fater_Lee

b. Now load them into variables:

In [ ]:
song1 = wavReadMono("song1.wav")
song2 = wavReadMono("song2.wav")

In [ ]:
#Listen to them if you'd like:
play(song1)
play(song2)

# Part 1: Filtering out an alarm sound #

a. In this section, we'll simulate a simple alarm sound being recorded over the music in song1. Then, you'll create a filter to remove this alarm sound as best you can.

In [ ]:
#This code adds a 5000Hz sine wave to the first 1 second of song1, then stores it in a new variable:
t = np.arange(0, 1, 1/44100)
song1_5000 = song1[0:44100] + 0.3 * sin(2*pi*5000*t)
play(song1) #normal song
play(song1_5000) #song with alarm

Listen to the normal song, and the song with the alarm added, above. Make sure you can hear the alarm.

b. The `signal.firwin` function will create you a new filter. At minimum, you need to tell it how many "taps" it has, and the cutoff frequency (or frequencies, in the case of a bandpass/bandstop filter).

The number of taps is another way of saying the number of filter coefficients, which is the same thing as the length of its impulse response). This could be anywhere from 1 to thousands. 

The cutoff frequency is expressed as a proportion of the Nyquist rate. For instance, a cutoff of 0.5 would be 1/2 the Nyquist rate, which would correspond to 11025Hz if you have a sample rate of 44100Hz (and therefore a Nyquist rate of 22050 Hz).

Use the `?` syntax below to read more about this filter design function, and scroll down in the help file to see examples of how to build different filter types (e.g., low-pass, high-pass, band-pass, band-stop).

In [1]:
?signal.firwin

Choose one of the examples from this help file and run it below. Instead of printing the output of this function call to the screen, though, save it in a variable called `b`. (This is the filter's impulse response, which is the same as its array of filter coefficients.)

In [ ]:
#Add the example code here, storing the function output in b:
b = 

The function `signal.freqz` can be used to plot the frequency response of any filter. You can read about it using the `?` :

In [ ]:
?signal.freqz

The code below is wraps up the example code at the end of the `signal.freqz` help file, and puts it in one function called `plotResponse`. Once you run the cell below, you can call `plotResponse` to plot the frequency response of any filter coefficeint array. The x-axis is frequency, from 0 to the Nyquist rate (which is pi radians per sample). On the y-axis, you will see the *magnitude response* in blue and the *phase response* in green. 

In [ ]:
def plotResponse(myCoeffs):
    w, h = signal.freqz(myCoeffs) #get frequency response for h
    fig = plt.figure()
    plt.title('Digital filter frequency response')
    ax1 = fig.add_subplot(111)

    plt.plot(w, 20 * np.log10(abs(h)), 'b') #plot in dB scale, wow!
    plt.ylabel('Amplitude [dB]', color='b')
    plt.xlabel('Frequency [rad/sample]')

    ax2 = ax1.twinx()
    angles = np.unwrap(np.angle(h))
    plt.plot(w, angles, 'g')
    plt.ylabel('Angle (radians)', color='g')
    plt.grid()
    plt.axis('tight')
    plt.show()

Use the code below to plot the frequency response for your new filter array, `b`:

In [ ]:
plotResponse(b)

Does the frequency response you've plotted match your expectation, based on how you computed `b`? If no, revisit the help for `?signal.firwin` where you found the example code to compute `b`, or ask for help.

c. Now, let's design a filter to remove the 5000Hz alarm sound from `song1_5000`. What type of filter should you use?


Write your answer here (double-click to edit)

What frequency/frequencies should you use for the cutoff of this filter? What is it / what are they, as a proportion of the Nyquist rate (22050Hz)?

Write your answer here (double-click to edit)

Using the `?signal.firwin` help as needed, compute a new set of filter coefficients to filter out this unwanted sine wave. Then plot the frequency response to verify that this filter will do what you want.

In [ ]:
b = ? #compute new coefficients...
plotResponse(b)

Convolve `b` with `song1_5000` and listen to the result. If you've done everything right, you should hear the music but no annoying alarm sound!

d. Experiment with changing the numtaps and listen to how the filtered sound is changed. How many taps do you need, at minimum, to get a good sonic result? Is there anything bad that happens as the number of taps becomes very large?
Show your experimentation and your written answers to these questions below, creating more cells as needed.

# Part 2: Isolating the bass drum #

There are many reasons you may want to try to isolate a single instrument from a piece of music. You may want a clean sample that you can remix in other contexts. Or you may want to use that instrument's behaviour to say something useful about the music.

For example, drum sounds can be good indicators of the tempo of a piece of music. A drum often repeats its pattern every 2 or 4 measures within a section of music; knowing when each drum pattern begins can therefore help you segment music into meaningful sections. Different music genres also have characteristic drum beats, so knowing when drums play can help you identify genre.

In this section, design a filter that you can apply to song2 to remove everything but the bass drum.

In [ ]:
#Your work goes here. Use the filter design tool, freqz, and your own listening to design a good filter.














# Part 3: Subtractive synthesis #

In subtractive synthesis, we apply a filter to a sound with a rich spectrum (such as white noise, an impulse, or a pulse train). In this section, experiment with using different filters applied to noise to get a feel for what types of filters are useful, and what type of sounds you can make.

The following code generates white noise for as many samples as you want:

In [ ]:
numSamples = 44100
noise = (rand(numSamples)-0.5)*2 #values will be between [-1, 1]

Try creating a bandpass filter with a narrow band around 1000 Hz. When you apply it to noise, what do you hear?

In [ ]:
numTaps = 500
f1 = 950/44100
f2 = 1050/44100
b = signal.firwin(numTaps, [f1, f2], pass_zero=False)
plotResponse(b)
play(convolve(b, noise))

Use the space below to experiment with how the sound changes as you change the number of taps and the bandwidth (i.e., difference between the two filter cutoff frequencies). Make a note of sounds that you like.

In [ ]:
#your work goes here








